In [ ]:
import librosa
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display
import os

# Sæt matplotlib til at vise plots inline
plt.rcParams['figure.figsize'] = (12, 6)
print("Biblioteker indlæst succesfuldt!")

# OPGAVE 2: Kvantisering og Dithering

I denne opgave vil vi:
1. Indlæse et klaversignal (60.flac)
2. Genkvantisere signalet til 6 bits opløsning ved afrunding
3. Lytte til og beskrive resultatet
4. Tilføje dither-støj med trekantet fordeling før kvantisering
5. Sammenligne de forskellige resultater

In [ ]:
# Indlæs klaversignalet
piano_file = "60.flac"

print(f"Indlæser klaverfil: {piano_file}")
if os.path.exists(piano_file):
    # Indlæs audio signalet
    y_piano, sr_piano = librosa.load(piano_file, sr=None)
    
    print(f"Sample rate: {sr_piano} Hz")
    print(f"Varighed: {len(y_piano)/sr_piano:.2f} sekunder")
    print(f"Signal form: {y_piano.shape}")
    print(f"Min værdi: {np.min(y_piano):.6f}")
    print(f"Max værdi: {np.max(y_piano):.6f}")
    
    # Normaliser signalet til [-1, 1] range hvis nødvendigt
    max_val = np.max(np.abs(y_piano))
    if max_val > 1.0:
        y_piano = y_piano / max_val
        print(f"Signal normaliseret (divideret med {max_val:.3f})")
    
else:
    print(f"Filen {piano_file} blev ikke fundet!")
    print("Opretter et kunstigt klaversignal til demonstration...")
    
    # Skab et simpelt syntetisk klaversignal som backup
    sr_piano = 22050
    duration = 5  # sekunder
    t = np.linspace(0, duration, int(sr_piano * duration))
    
    # Grundfrekvens (A4 = 440 Hz)
    fundamental = 440
    
    # Skab en klaver-lignende tone med harmoniske og decay
    y_piano = (np.sin(2 * np.pi * fundamental * t) * 0.6 +
               np.sin(2 * np.pi * fundamental * 2 * t) * 0.3 +
               np.sin(2 * np.pi * fundamental * 3 * t) * 0.1) * np.exp(-t * 0.8)
    
    print("Syntetisk klaversignal oprettet")

In [ ]:
# Kvantiseringsfunktioner
def quantize_signal(signal, bits=6):
    """
    Kvantiser signal til specificeret antal bits ved afrunding
    
    Args:
        signal: Input signal (normaliseret til [-1, 1])
        bits: Antal bits til kvantisering
    
    Returns:
        Kvantiseret signal
    """
    # Beregn antal kvantiseringsniveauer
    levels = 2**bits
    
    # Skalér signal til kvantiseringsniveauer
    scaled = signal * (levels // 2 - 1)
    
    # Afrund til nærmeste heltal
    quantized_int = np.round(scaled)
    
    # Skalér tilbage til [-1, 1]
    quantized = quantized_int / (levels // 2 - 1)
    
    return quantized

def generate_triangular_dither(length, amplitude=1.0):
    """
    Generer trekantet dither-støj ved at addere to uniforme støjsignaler
    
    Args:
        length: Antal samples
        amplitude: Amplitude af dither (typisk 2 kvantiseringsstrin)
    
    Returns:
        Trekantet dither-støj
    """
    # Generer to uniforme støjsignaler [-0.5, 0.5]
    noise1 = np.random.uniform(-0.5, 0.5, length)
    noise2 = np.random.uniform(-0.5, 0.5, length)
    
    # Addér dem for at få trekantet fordeling
    triangular_noise = (noise1 + noise2) * amplitude
    
    return triangular_noise

def quantize_with_dither(signal, bits=6):
    """
    Kvantiser signal med trekantet dither
    
    Args:
        signal: Input signal
        bits: Antal bits til kvantisering
    
    Returns:
        Kvantiseret signal med dither
    """
    # Beregn kvantiseringsstørrelse
    levels = 2**bits
    quantization_step = 2.0 / levels  # For [-1, 1] range
    
    # Generer trekantet dither der dækker 2 kvantiseringsstrin
    dither_amplitude = quantization_step
    dither = generate_triangular_dither(len(signal), dither_amplitude)
    
    # Tilføj dither til signal
    signal_with_dither = signal + dither
    
    # Kvantiser
    quantized = quantize_signal(signal_with_dither, bits)
    
    return quantized

# Test kvantisering på klaversignal
bits = 6
print(f"Kvantiserer klaversignal til {bits} bits...")

# Kvantiser uden dither
y_quantized = quantize_signal(y_piano, bits)

# Kvantiser med dither
y_quantized_dither = quantize_with_dither(y_piano, bits)

print(f"Original signal RMS: {np.sqrt(np.mean(y_piano**2)):.6f}")
print(f"Kvantiseret (uden dither) RMS: {np.sqrt(np.mean(y_quantized**2)):.6f}")
print(f"Kvantiseret (med dither) RMS: {np.sqrt(np.mean(y_quantized_dither**2)):.6f}")

# Beregn kvantiseringsstøj
quantization_noise = y_piano - y_quantized
quantization_noise_dither = y_piano - y_quantized_dither

print(f"Kvantiseringsstøj RMS (uden dither): {np.sqrt(np.mean(quantization_noise**2)):.6f}")
print(f"Kvantiseringsstøj RMS (med dither): {np.sqrt(np.mean(quantization_noise_dither**2)):.6f}")

In [ ]:
# Visualisering af kvantiseringseffekter
def plot_quantization_comparison(original, quantized, quantized_dither, start_time=1, duration=0.5):
    """
    Sammenlign original, kvantiseret uden dither og kvantiseret med dither
    """
    start_sample = int(start_time * sr_piano)
    end_sample = start_sample + int(duration * sr_piano)
    
    time = np.arange(start_sample, end_sample) / sr_piano
    
    fig, axes = plt.subplots(3, 1, figsize=(14, 10))
    
    # Original signal
    axes[0].plot(time, original[start_sample:end_sample], 'b-', linewidth=1)
    axes[0].set_title('Original klaversignal')
    axes[0].set_ylabel('Amplitude')
    axes[0].grid(True, alpha=0.3)
    axes[0].set_ylim(-1.1, 1.1)
    
    # Kvantiseret uden dither
    axes[1].plot(time, quantized[start_sample:end_sample], 'r-', linewidth=1)
    axes[1].set_title(f'Kvantiseret til {bits} bits (uden dither)')
    axes[1].set_ylabel('Amplitude')
    axes[1].grid(True, alpha=0.3)
    axes[1].set_ylim(-1.1, 1.1)
    
    # Kvantiseret med dither
    axes[2].plot(time, quantized_dither[start_sample:end_sample], 'g-', linewidth=1)
    axes[2].set_title(f'Kvantiseret til {bits} bits (med trekantet dither)')
    axes[2].set_xlabel('Tid (s)')
    axes[2].set_ylabel('Amplitude')
    axes[2].grid(True, alpha=0.3)
    axes[2].set_ylim(-1.1, 1.1)
    
    plt.tight_layout()
    plt.show()

# Plot sammenligning
print("Sammenligning af kvantiseringsmetoder:")
plot_quantization_comparison(y_piano, y_quantized, y_quantized_dither)

In [ ]:
# Frekvensanalyse af kvantiseringseffekter
def plot_quantization_spectrum_comparison(original, quantized, quantized_dither):
    """
    Sammenlign frekvensspektre for kvantiseringsmetoder
    """
    # Beregn FFT
    fft_original = np.fft.fft(original)
    fft_quantized = np.fft.fft(quantized)
    fft_quantized_dither = np.fft.fft(quantized_dither)
    
    # Frekvensvektor
    freqs = np.fft.fftfreq(len(original), 1/sr_piano)
    
    # Tag kun positive frekvenser
    n = len(freqs) // 2
    freqs = freqs[:n]
    
    # Magnitude spektre
    mag_original = np.abs(fft_original[:n])
    mag_quantized = np.abs(fft_quantized[:n])
    mag_quantized_dither = np.abs(fft_quantized_dither[:n])
    
    fig, axes = plt.subplots(3, 1, figsize=(14, 12))
    
    # Original spektrum
    axes[0].semilogx(freqs[1:], 20*np.log10(mag_original[1:] + 1e-10), 'b-', alpha=0.7)
    axes[0].set_title('Frekvensspektrum - Original klaversignal')
    axes[0].set_ylabel('Magnitude (dB)')
    axes[0].grid(True, alpha=0.3)
    axes[0].set_xlim(20, sr_piano/2)
    
    # Kvantiseret uden dither spektrum
    axes[1].semilogx(freqs[1:], 20*np.log10(mag_quantized[1:] + 1e-10), 'r-', alpha=0.7)
    axes[1].set_title(f'Frekvensspektrum - Kvantiseret til {bits} bits (uden dither)')
    axes[1].set_ylabel('Magnitude (dB)')
    axes[1].grid(True, alpha=0.3)
    axes[1].set_xlim(20, sr_piano/2)
    
    # Kvantiseret med dither spektrum
    axes[2].semilogx(freqs[1:], 20*np.log10(mag_quantized_dither[1:] + 1e-10), 'g-', alpha=0.7)
    axes[2].set_title(f'Frekvensspektrum - Kvantiseret til {bits} bits (med dither)')
    axes[2].set_xlabel('Frekvens (Hz)')
    axes[2].set_ylabel('Magnitude (dB)')
    axes[2].grid(True, alpha=0.3)
    axes[2].set_xlim(20, sr_piano/2)
    
    plt.tight_layout()
    plt.show()

print("Frekvensanalyse af kvantiseringseffekter:")
plot_quantization_spectrum_comparison(y_piano, y_quantized, y_quantized_dither)

In [ ]:
# Lyt til kvantiseringsresultaterne
print("Audio afspilning - Kvantiseringssammenligning:")
print("=" * 60)

# Original klaversignal
print("Original klaversignal:")
display(Audio(y_piano, rate=sr_piano))

print("\n" + "=" * 60)

# Kvantiseret uden dither
print(f"Kvantiseret til {bits} bits (uden dither):")
display(Audio(y_quantized, rate=sr_piano))

print("\n" + "=" * 60)

# Kvantiseret med dither
print(f"Kvantiseret til {bits} bits (med trekantet dither):")
display(Audio(y_quantized_dither, rate=sr_piano))

print("\n" + "=" * 60)

# Kvantiseringsstøj
print("Kvantiseringsstøj (original - kvantiseret uden dither) - forstærket 10x:")
noise_amplified = (y_piano - y_quantized) * 10
display(Audio(noise_amplified, rate=sr_piano))

print("\n" + "=" * 60)

print("Kvantiseringsstøj (original - kvantiseret med dither) - forstærket 10x:")
noise_dither_amplified = (y_piano - y_quantized_dither) * 10
display(Audio(noise_dither_amplified, rate=sr_piano))

In [ ]:
# Demonstration af forskellige bit-dybder
print("Sammenligning af forskellige kvantiseringsdybder:")
print("=" * 50)

bit_depths = [3, 4, 6, 8]
quantized_signals = {}

for bits_demo in bit_depths:
    # Kvantiser uden dither
    q_no_dither = quantize_signal(y_piano, bits_demo)
    # Kvantiser med dither
    q_with_dither = quantize_with_dither(y_piano, bits_demo)
    
    quantized_signals[bits_demo] = {
        'no_dither': q_no_dither,
        'with_dither': q_with_dither
    }
    
    # Beregn SNR (Signal-to-Noise Ratio)
    noise_no_dither = y_piano - q_no_dither
    noise_with_dither = y_piano - q_with_dither
    
    snr_no_dither = 20 * np.log10(np.sqrt(np.mean(y_piano**2)) / np.sqrt(np.mean(noise_no_dither**2)))
    snr_with_dither = 20 * np.log10(np.sqrt(np.mean(y_piano**2)) / np.sqrt(np.mean(noise_with_dither**2)))
    
    print(f"{bits_demo} bits:")
    print(f"  SNR uden dither: {snr_no_dither:.1f} dB")
    print(f"  SNR med dither:  {snr_with_dither:.1f} dB")
    print()

# Lyt til forskellige bit-dybder (kun 3 og 6 bits for sammenligning)
print("Lyt til forskellige kvantiseringsdybder:")
print("=" * 50)

for bits_demo in [3, 6]:
    print(f"\n{bits_demo} bits uden dither:")
    display(Audio(quantized_signals[bits_demo]['no_dither'], rate=sr_piano))
    
    print(f"{bits_demo} bits med dither:")
    display(Audio(quantized_signals[bits_demo]['with_dither'], rate=sr_piano))

# Gem filer til sammenligning
print("\nGemmer audio filer...")
sf.write("piano_original.wav", y_piano, sr_piano)
sf.write("piano_6bit_no_dither.wav", y_quantized, sr_piano)
sf.write("piano_6bit_with_dither.wav", y_quantized_dither, sr_piano)
print("Filer gemt: piano_original.wav, piano_6bit_no_dither.wav, piano_6bit_with_dither.wav")